In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_classes=2, n_informative=2,
                           n_redundant=0, random_state=42)
y = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Classifier from scratch
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature            # index of feature to split
        self.threshold = threshold        # threshold value to split
        self.left = left                  # left child node
        self.right = right                # right child node
        self.value = value                # class label if leaf

    def is_leaf(self):
        return self.value is not None

class DecisionTreeClassifierScratch:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))

        # Stopping criteria
        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._majority_class(y)
            return Node(value=leaf_value)

        # Find best split
        best_feat, best_thresh = self._best_split(X, y)

        if best_feat is None:
            leaf_value = self._majority_class(y)
            return Node(value=leaf_value)

        # Split the dataset
        left_idx = X[:, best_feat] < best_thresh
        right_idx = ~left_idx

        left = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return Node(feature=best_feat, threshold=best_thresh, left=left, right=right)

    def _gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        prob_sq = (counts / len(y)) ** 2
        return 1 - np.sum(prob_sq)

    def _best_split(self, X, y):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feature_idx in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_idx])
            for t in thresholds:
                left_idx = X[:, feature_idx] < t
                right_idx = ~left_idx

                if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
                    continue

                # Gini impurity
                gini_left = self._gini(y[left_idx])
                gini_right = self._gini(y[right_idx])
                weighted_avg = (len(y[left_idx]) * gini_left + len(y[right_idx]) * gini_right) / len(y)

                gain = self._gini(y) - weighted_avg

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_thresh = t

        return split_idx, split_thresh

    def _majority_class(self, y):
        counts = np.bincount(y.flatten())
        return np.argmax(counts)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value

        if x[node.feature] < node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

tree = DecisionTreeClassifierScratch(max_depth=3)
tree.fit(X_train, y_train)
preds = tree.predict(X_test)
accuracy = np.mean(preds.reshape(-1, 1) == y_test)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.85
